In [7]:
!pip install htmlmin openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.2 MB/s eta 0:00:00


In [8]:
import requests
from bs4 import BeautifulSoup, Comment
from google.oauth2 import service_account
import gspread
import pandas as pd
import htmlmin
import openai
import json

In [9]:
openai.api_key = "..."

In [10]:
# https://docs.google.com/spreadsheets/d/1GNYZVseLnpxQonekssoTsP92IuHGkBlHUIgS7xzXR8E
csv_url = 'https://testbucket1841.s3.ap-south-1.amazonaws.com/csv-dump/nptel-data.csv'

# Load the CSV data
data = pd.read_csv(csv_url)
data = data.fillna('')
data.head(5)

,Course Id,Discipline,Course Name,SME Name,Institute,\nCo-ordinating Institute,Duration,Type of course,Start date,End date,...,Enrollment End date,UG/PG,Core/Elective,FDP,Applicable NPTEL Domain,Click here to join the course,Old course URL,NPTEL URL,Old course URL.1,NPTEL URL.1
0,noc23-ae01,Aerospace Engineering,Aircraft Design,Prof. A.K.Ghosh,IITK,IITK,12 Weeks,Rerun,"January 23, 2023","April 14, 2023",...,"January 30, 2023",UG/PG,Core,Yes,Flight Mechanics,https://onlinecourses.nptel.ac.in/noc23_ae01,https://onlinecourses.nptel.ac.in/noc22_ae01,https://nptel.ac.in/courses/101104069,https://onlinecourses.nptel.ac.in/noc22_ae01,https://nptel.ac.in/courses/101104069
1,noc23-ae02,Aerospace Engineering,Introduction to Experiments in Flight,Prof. A.K.Ghosh,IITK,IITK,4 Weeks,Rerun,"January 23, 2023","February 17, 2023",...,"January 30, 2023",UG,Core,No,Flight Mechanics,https://onlinecourses.nptel.ac.in/noc23_ae02,https://onlinecourses.nptel.ac.in/noc22_ae02,https://nptel.ac.in/courses/101104066,https://onlinecourses.nptel.ac.in/noc22_ae02,https://nptel.ac.in/courses/101104066
2,noc23-ae03,Aerospace Engineering,Rocket Propulsion,Prof. K. Ramamurthi \n Prof. V. Nagabhushana Rao,IITM,IITM,12 Weeks,Rerun,"January 23, 2023","April 14, 2023",...,"January 30, 2023",UG/PG,Elective,Yes,Propulsion,https://onlinecourses.nptel.ac.in/noc23_ae03,https://onlinecourses.nptel.ac.in/noc22_ae03,https://nptel.ac.in/courses/101106082,https://onlinecourses.nptel.ac.in/noc22_ae03,https://nptel.ac.in/courses/101106082
3,noc23-ae04,Aerospace Engineering,Computational Science in Engineering,Prof. Ashok De,IITK,IITK,8 Weeks,Rerun,"January 23, 2023","March 17, 2023",...,"January 30, 2023",UG/PG,Core/Elective,Yes,,https://onlinecourses.nptel.ac.in/noc23_ae04,https://onlinecourses.nptel.ac.in/noc22_ae04,https://nptel.ac.in/courses/101104086,https://onlinecourses.nptel.ac.in/noc22_ae04,https://nptel.ac.in/courses/101104086
4,noc23-ae05,Aerospace Engineering,Lighter than Air Systems,Prof. Rajkumar S. Pant,IITB,IITB,12 Weeks,Rerun,"January 23, 2023","April 14, 2023",...,"January 30, 2023",UG/PG,Elective,Yes,,https://onlinecourses.nptel.ac.in/noc23_ae05,https://onlinecourses.nptel.ac.in/noc22_ae07,https://nptel.ac.in/courses/101101087,https://onlinecourses.nptel.ac.in/noc22_ae07,https://nptel.ac.in/courses/101101087


In [11]:
(print(data.columns))

Index(['Course Id', 'Discipline', 'Course Name', 'SME Name', 'Institute',
       '\nCo-ordinating Institute', 'Duration', 'Type of course', 'Start date',
       'End date', 'Exam date', 'Enrollment End date', 'UG/PG',
       'Core/Elective', 'FDP', 'Applicable NPTEL Domain',
       'Click here to join the course', 'Old course URL', 'NPTEL URL',
       'Old course URL.1', 'NPTEL URL.1'],
      dtype='object')


In [12]:
# Read course URLs from Google Sheet
urls = data.iloc[:,16]
urls.head()

0    https://onlinecourses.nptel.ac.in/noc23_ae01
1    https://onlinecourses.nptel.ac.in/noc23_ae02
2    https://onlinecourses.nptel.ac.in/noc23_ae03
3    https://onlinecourses.nptel.ac.in/noc23_ae04
4    https://onlinecourses.nptel.ac.in/noc23_ae05
Name: Click here to join the course, dtype: object

In [13]:
from builtins import str

In [14]:
auth_token="g_a=820k7bwIBtvWUFPzbkvtnHXZL8tG6nn6wKGtC7dG8K88RrO4AxIyLh8XuG18v9wa; g_b=poC8ll+QvTXgYYHFuF9liOQpdLn/ji89TDzlfI/xfuC7QrLLh8tIgnckL1RGrxTLfOImCqIoyQxm2ewccnFl/6LdJf9ftY9RxdyHCq/iJFw=; g_c=vhrba5TuVTzoMqPY/6ev0EM/3w36y7kv+uI9EjjODuN96CH45Ua8Hs2mwVnvY94T/ou8eDg3PIRrrs0juToSGOecYwCZ0o1oM2V9AA10r9nH+fuRtZ8bm8FV2gvsPnKPooHtJlPGiXe/wgff46ddNLM025uR9wXFPSe/s0EwF0xblKdY0ExzpCX+oomFk2uEYIcyMRbE2iZg5GkuDqscFtY5iLEnqIhOiNQ1PJCaoyfiF3GH4K0BIlo/6zu1kXza0uX6jmORw36esiX6UGjNNvlnCSNNSt1nuf281AmG8/gx/k4Dx9GjICn01pQ+38umRtq5RJRWsLCO98PSHc+eXF/bIE4IG8GhBpPdizZK9/Q/JiMagbTTfBsZsfULygxTYHBvv//OPtArdjkA+dfMS95TjM2SFmsY/OajxIDrHMAdg8YG9GKa7lqSAWAYKD0PvmPF4lQDEMO5TM1tSG859qDH19wrd3NHAgpCmJggBNmu3eTeS0t53t1PT4CB5W6qEg9A+bCVOOrScCibHYLN39xGXuAwGVxO+8JIL27XUYe6EBhCjInlvEdZ5qYDkF9ZDlSRDDz5bC/Dekx5pHc3ay3rlcS88nYkaoWjMyL1NJn1biHkNxjqQ3da40hmWtXe8b3HSufJjJLITE1ANDy3e0+7Mc+YBYcBAmPvkFF1tjTiP9St6RJ0Mn2Iwx3VKSwxNQoPgbIYhHKDrcEpzBZGGZsVDrVgg/Y5kITEj8/TlUbzfhIcdx4FnbrmpvF86KWdGMKUmuN/+iXbMH/Ho1xCnMVRMAf0pn55vfKxCPV39ax1V6/2Eb87dMCXNwCLVwF33wCvuhldGPlGBhS3Jqvq/XiN1v2A3c66GSHJVdhkb1BML8Z/PeBKD0NQ+zF/biOw4XKDNRUcXjEg6vPh9djqdjVbz1iO/H6PQyLt5E/Lrd08/c7wEhTs787MAqHWYOjfimERb4jwH5xmkPYUduudJWK/XWMc1YhqukbU40qzUHrp0onCtYAoVOKN1fCIdWOk84XnVIUIn4RmIiH5bRU887nTIaCU/mxBH57sxiLaPvGJH73w+snMyJ0Tode9AU6u2jUOsa31/KSDNh2r3Qo6QDgs48wepVi3E1JrwN1r2F7jd6zRb27ItNmPK7lNXq0zOTIemEU+T/glOERhV/vu7ppDzg+nwd24TE99Ct4h4WvQZPA/6sAIqkFlZ/cdLus1Yxnm64INHN0R6suJQHx8+BVIQvOAbIeFZjUQZYDmXn5mdkMOJ8sRYQnBWQM0lK3Ge4nbYqVU1yPHoo4+bq0XNxO7YeadKDlievMSJuPYlVdG0N4zJSkYlS/lnXLjjQOy; amp_adc4c4=TZzwzz4je6WzlnQ9OHCTUJ...1gvqmhjff.1gvqmhkkr.0.0.0"
headers = {"Authorization": f"Bearer {auth_token}"}

def find_text(soup, keyword):
    keyword_lower = keyword.lower()
    element = soup.find(lambda tag: tag.name ==
                        'span' and keyword_lower in tag.text.lower())
    return element.text if element else ''


# Function to scrape data from course page
def scrape_data(url,usage_token_count):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print('=> 2.Received HTML data for url.... ⏰')
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

    # with open('response.txt', 'w') as file:
    #     file.write(response.text)
    # div_elements = soup.find_all('div', class_='yui-wk-div')
    # print(div_elements)
    # for div in div_elements:
    #     # Process the div element as needed
    #     print(div.prettify()

    print('=> 3.Cleaning and processing HTML using beautiful soup 🍜')
    soup = BeautifulSoup(response.text, 'html.parser')
    # Remove all unecessary tags
    for tag in soup(['script', 'style','link','head','img','button','option','a','br','footer','section','header','table','tr','td','select']):
        tag.decompose()
    # Remove all comments from the HTML
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()
    # remove all class attributes
    for tag in soup.find_all():
        tag.attrs = {key: value for key, value in tag.attrs.items() if key not in ['class', 'id','style','onclick']}
    # remove all empty tags
    for tag in soup.find_all():
        if not tag.contents and not tag.name == 'br':
            tag.extract()
    
    # Get the updated HTML
    clean_html = str(soup)
    minified_html = htmlmin.minify(clean_html, remove_empty_space=True)
    html_string = minified_html.replace('"', "'")

    print('=> 4.Asking openAI to do cool completion stuff now 🤖')
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    messages=[
            {"role": "user", "content": f"I have the following extracted HTML code for a webpage containing information about an online course ```{html_string}``` Kindly extract values pertaining to 'intended audience', 'industry support', 'youtube video url','course_layout' and 'prerequisites' for this course, from this HTML data. The 'course_layout' information should be an array of weekly topics covered. Return ONLY a JSON object of the following form containing the relevant information for the keys:object:{{title, instructorName, instructorBio, youtubeUrl, prerequisites, intendedAudience, industrySupport, courseLayout}}. Return nothing else in response except a JSON object"},
        ]
    )
    print('=> 5.Received response from AI agent 😏 - ')
    usage_token_count += response.usage.total_tokens
    with open('processing.log', 'a') as f:
        f.write(f"Processed {url} with {response.usage.total_tokens} - Prompt({response.usage.prompt_tokens}) and Completion({response.usage.completion_tokens}). Total token usage Count - {usage_token_count} \n")
    dataJsonString = response.choices[0].message.content.replace("\\", "").replace("\n", "")
    dataJson = json.loads(dataJsonString)

    print('=> 6.Formatted AI response object now 💭')
    infoObj = {
        'title': dataJson['title'],
        'instructor_name': dataJson['instructorName'],
        'instructor_bio': dataJson['instructorBio'],
        'youtube_url': dataJson['youtubeUrl'],
        'prerequisites': dataJson['prerequisites'],
        'intended_audience':  dataJson['intendedAudience'],
        'industry_support': dataJson['industrySupport'],
        'course_layout': dataJson['courseLayout'],
        
    }
    return infoObj

    ## Older approach using beautiful soup search and filtering - did not work 
    # with open('html.txt', 'w') as file:
    #     file.write(html_string)
    # intended_audience = find_text(soup,'intended audience')
    # industry_support = find_text(soup,'industry support')
    # prerequisites = find_text(soup,'prerequisites')

    # youtube_url = ''
    # youtube_element = soup.find(
    #     'iframe', src=lambda x: x and 'youtube.com' in x)
    # if youtube_element:
    #     youtube_url = youtube_element['src']


    # return intended_audience, industry_support, prerequisites, youtube_url
# With this change, the script will now search for the specified keywords within 'span' elements on the HTML page.


In [15]:
# Maintain an active count of tokens being used for OpenAI API
usage_token_count = 0

def process_csv_chunk(chunk):
    urls = chunk.iloc[:,16]
    urls.head()
    with open('error.log', 'w') as f:
        for i, url in enumerate(urls):
            url = url + '/preview'
            try:
                # Make an API request using the value
                print(f"--- PERFORMING OPERATION ON ROW {i}")
                print(f"=> 1. Scraping data from url - {url}")
                scraped_info = scrape_data(url,usage_token_count);
                print("Here's the motherfucking data - ", scraped_info,"🔥")
                data.at[i, 'intended_audience'] = scraped_info['intended_audience']
                data.at[i, 'instructor_name'] = scraped_info['instructor_name']
                data.at[i, 'instructor_bio'] = scraped_info['instructor_bio']
                data.at[i, 'course_layout'] = ';'.join(scraped_info['course_layout'])
                data.at[i, 'prerequisites'] = scraped_info['prerequisites']
                data.at[i, 'industry_support'] = scraped_info['industry_support']
                data.at[i, 'course_url'] = url
            except Exception as e:
                # Write the error to the log file
                f.write(f'Error processing value {url}: {str(e)}\n')
                continue
            print(f'✅ Successfully scraped url' + '\n' + '----------' + '\n')
    return chunk
# Open a log file for writing errors


In [16]:
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

#### Setting up base for parallelizing operations over this CSV data
def parallelize_dataframe_processing(df, func, num_partitions, num_workers):
    chunk_size = len(df) // num_partitions
    chunks = [df.iloc[i * chunk_size: (i + 1) * chunk_size] for i in range(num_partitions)]

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        processed_chunks = list(executor.map(func, chunks))

    return pd.concat(processed_chunks)

In [ ]:
file_path = csv_url
df = pd.read_csv(file_path)

num_partitions = 6
num_workers = 6

processed_df = parallelize_dataframe_processing(df, process_csv_chunk, num_partitions, num_workers)

# Save the processed DataFrame to a new CSV file
processed_df.to_csv('processed_file.csv', index=False)

--- PERFORMING OPERATION ON ROW 0
=> 1. Scraping data from url - https://onlinecourses.nptel.ac.in/noc23_ae01/preview
--- PERFORMING OPERATION ON ROW 0--- PERFORMING OPERATION ON ROW 0

--- PERFORMING OPERATION ON ROW 0--- PERFORMING OPERATION ON ROW 0=> 1. Scraping data from url - https://onlinecourses.nptel.ac.in/noc23_cs42/preview=> 1. Scraping data from url - https://onlinecourses.nptel.ac.in/noc23_cy16/preview


--- PERFORMING OPERATION ON ROW 0=> 1. Scraping data from url - https://onlinecourses.nptel.ac.in/noc23_ee73/preview
=> 1. Scraping data from url - https://onlinecourses.nptel.ac.in/noc23_mg39/preview


=> 1. Scraping data from url - https://onlinecourses.nptel.ac.in/noc23_me37/preview
=> 2.Received HTML data for url.... ⏰
=> 3.Cleaning and processing HTML using beautiful soup 🍜
=> 4.Asking openAI to do cool completion stuff now 🤖
=> 2.Received HTML data for url.... ⏰
=> 3.Cleaning and processing HTML using beautiful soup 🍜
=> 4.Asking openAI to do cool completion stuff no